# AWS IoT Greengrass Model Component Setup for DDA

This notebook automates the creation of AWS IoT Greengrass model components for the Defect Detection Application (DDA). The process includes:

1. **Model Artifact Processing**: Download and prepare trained model artifacts
2. **Directory Structure Setup**: Organize files for Greengrass deployment
3. **Component Creation**: Generate and deploy Greengrass model components

## Prerequisites
- Trained models from SageMaker (both trained and compiled)
- AWS CLI configured with appropriate permissions
- Access to AWS IoT Greengrass v2 service

## Workflow Overview

The Greengrass model component creation involves three main phases:

### Phase 1: Model Artifact Preparation
- Download trained model artifacts from S3
- Extract and analyze model configuration
- Create DDA-compatible manifest file

### Phase 2: Directory Structure Setup
- Download compiled model artifacts
- Organize files in Greengrass-compatible structure
- Package and upload to S3

### Phase 3: Component Creation
- Generate Greengrass component recipe
- Create and validate the component
- Monitor deployment status

In [ ]:
# Import required dependencies
import os
import tarfile
import boto3
import yaml
import json
import shutil
import uuid
import traceback
import re
import time
import sys

# Visual indicators for status messages
green_check_mark = "\u2705"  # ✅ Success indicator
error_mark = "\u274C"       # ❌ Error indicator

def print_tree(dir_path, prefix=""):
    """
    Display directory structure in tree format for better visualization.
    
    Args:
        dir_path (str): Path to directory to display
        prefix (str): Prefix for tree formatting
    """
    items = os.listdir(dir_path)
    
    for index, item in enumerate(items):
        # Format tree structure
        if index == len(items) - 1:
            print(prefix + "└── " + item)
            new_prefix = prefix + "    "
        else:
            print(prefix + "├── " + item)
            new_prefix = prefix + "│   "
        
        # Recursively display subdirectories
        item_path = os.path.join(dir_path, item)
        if os.path.isdir(item_path):
            print_tree(item_path, new_prefix)

print(f"{green_check_mark} Environment setup complete")

---

## Phase 1: Model Artifact Preparation

Download and process the trained model artifacts to create DDA-compatible components.

### Step 1.1: Specify Model Artifact Locations

Provide S3 URIs for both trained (original) and compiled model artifacts.
💡 Note: The trained model URI can be found in SageMaker AI -> Training → Training jobs → [Your Job Name] → Output section
💡 Note: The compiled model URI can be found in SageMaker AI -> Inference -> Compilation jobs -> [Your Job Name] -> Output

In [ ]:
# Get S3 locations for model artifacts
print("📥 Model Artifact Configuration")
print("Please provide S3 URIs for your trained models:\n")

trained_model_artifacts = input("Enter S3 URI for trained model (from SageMaker training): ")
compiled_model_artifacts = input("Enter S3 URI for compiled model (from SageMaker Inference -> Compilation): ")

print(f"\n{green_check_mark} Model locations configured:")
print(f"  Trained: {trained_model_artifacts}")
print(f"  Compiled: {compiled_model_artifacts}")

### Step 1.2: Setup Working Directory

Create a clean workspace for model transformation and component preparation.

In [ ]:
# Setup working directory structure
current_dir = os.getcwd()
model_transformation_dir = os.path.join(current_dir, "model_transformation")

try:
    # Check if directory already exists
    if os.path.isdir(model_transformation_dir):
        print(f"{error_mark} Directory {model_transformation_dir} already exists.")
        
        # Prompt for cleanup confirmation
        response = input(f"\nDelete existing directory? (yes/no): ").strip().lower()
        if response == 'yes':
            shutil.rmtree(model_transformation_dir)
            print(f"{green_check_mark} Cleaned up existing directory")
        else:
            raise Exception("Directory cleanup declined")
    
    # Create fresh directory structure
    print("\n📁 Creating workspace directory...")
    os.makedirs(model_transformation_dir, exist_ok=True)
    
    # Setup subdirectories
    extract_dir = os.path.join(model_transformation_dir, 'model_folder')
    model_file_name = trained_model_artifacts.split('/')[-1]
    local_tar_dir = os.path.join(model_transformation_dir, model_file_name)
    
    print(f"{green_check_mark} Workspace created at: {model_transformation_dir}")
    
except Exception as e:
    print(f"\n{error_mark} Setup failed: {str(e)}")
    print("Please clean the directory manually and retry.")

### Step 1.3: Download Trained Model Artifacts

Retrieve the original trained model from S3 for processing.

In [ ]:
# Download trained model artifacts from S3
print("📥 Downloading trained model artifacts...")
!aws s3 cp {trained_model_artifacts} {model_transformation_dir}
print(f"{green_check_mark} Download completed")

### Step 1.4: Extract Model Archive

Extract the trained model to access configuration files and model artifacts.

In [ ]:
# Extract the trained model archive
os.makedirs(extract_dir, exist_ok=True)

try:
    print("📦 Extracting model archive...")
    with tarfile.open(local_tar_dir, 'r:gz') as tar:
        tar.extractall(path=extract_dir)
    
    print(f"{green_check_mark} Extraction completed at: {extract_dir}")
    print("\n📂 Extracted contents:")
    print_tree(extract_dir)
    
except Exception as e:
    print(f"{error_mark} Extraction failed: {str(e)}")
    traceback.print_exc()

### Step 1.5: Process Model Configuration

Read model configuration and manifest files to extract metadata needed for DDA integration.

In [ ]:
# Process model configuration and manifest files
config_path = os.path.join(extract_dir, 'config.yaml')
manifest_path = os.path.join(extract_dir, 'export_artifacts', 'manifest.json')
export_artifacts_dir = os.path.join(extract_dir, 'export_artifacts')

try:
    print("⚙️ Processing model configuration...")
    
    # Read dataset configuration from YAML
    with open(config_path, 'r') as yaml_file:
        config_data = yaml.safe_load(yaml_file)
        image_width = config_data['dataset']['image_width']
        image_height = config_data['dataset']['image_height']
    print(f"{green_check_mark} Configuration loaded - Image size: {image_width}x{image_height}")

    # Read model manifest
    with open(manifest_path, 'r') as json_file:
        manifest_data = json.load(json_file)
    print(f"{green_check_mark} Manifest loaded")
    
    # Locate PyTorch model file
    pt_file = None
    for file in os.listdir(export_artifacts_dir):
        if file.endswith('.pt'):
            pt_file = file
            break
    
    if not pt_file:
        raise FileNotFoundError(f"{error_mark} No PyTorch (.pt) model file found")
    
    print(f"{green_check_mark} Model file located: {pt_file}")

    # Extract model metadata
    input_shape = manifest_data.get('input_shape')
    model_type = manifest_data["model_graph"]["stages"][0]["type"]
    
    print(f"📊 Model Details:")
    print(f"  Type: {model_type}")
    print(f"  Input Shape: {input_shape}")
    print(f"  Image Dimensions: {image_width}x{image_height}")

    # Prepare compilable model configuration
    compilable_model = {
        "filename": pt_file,
        "data_input_config": {
            "input": input_shape
        },
        "framework": "PYTORCH"
    }
    
    # Dataset configuration for DDA
    dataset = {
        "image_width": image_width,
        "image_height": image_height
    }
    
except Exception as e:
    print(f"{error_mark} Configuration processing failed: {str(e)}")
    traceback.print_exc()

### Step 1.6: Generate DDA-Compatible Manifest

Create a new manifest file optimized for DDA deployment and edge inference.

In [ ]:
# Generate DDA-compatible manifest file
print("📝 Creating DDA-compatible manifest...")

# Construct new manifest with DDA-specific structure
new_manifest_data = {
    "model_graph": manifest_data["model_graph"],
    "compilable_models": [compilable_model],
    "dataset": dataset
}

# Extract model name from manifest
model_name = manifest_data["model_graph"]["stages"][0]["type"]
print(f"🏷️ Model type identified: {model_name}")

# Write the new manifest file
new_manifest_path = os.path.join(extract_dir, 'export_artifacts', 'new_manifest.json')
with open(new_manifest_path, 'w') as new_json_file:
    json.dump(new_manifest_data, new_json_file, indent=4)

print(f"{green_check_mark} DDA manifest created at: {new_manifest_path}")

In [ ]:
# Display the generated manifest for verification
print("📄 Generated DDA Manifest:")
print("=" * 50)
with open(new_manifest_path, 'r') as f:
    print(f.read())
print("=" * 50)

---

## Phase 2: Directory Structure Setup

Organize compiled model artifacts in the structure required by AWS IoT Greengrass components.

### Step 2.1: Download Compiled Model Artifacts

Retrieve the Neo-compiled model optimized for edge deployment.

In [ ]:
# Setup paths for compiled model processing
file_name = compiled_model_artifacts.split("/")[-1]
extract_dir_prefix = os.path.join(model_transformation_dir, 'model_artifacts')
compiled_extract_dir = os.path.join(extract_dir_prefix, model_name)
local_compiled_tar = os.path.join(model_transformation_dir, file_name)

print(f"📥 Downloading compiled model: {file_name}")
!aws s3 cp {compiled_model_artifacts} {model_transformation_dir}
print(f"{green_check_mark} Compiled model downloaded")

### Step 2.2: Extract Compiled Model

Extract the compiled model artifacts into the appropriate directory structure.

In [ ]:
# Extract compiled model artifacts
print("📦 Extracting compiled model artifacts...")
os.makedirs(compiled_extract_dir, exist_ok=True)

with tarfile.open(local_compiled_tar, 'r:gz') as tar:
    tar.extractall(path=compiled_extract_dir)

print(f"{green_check_mark} Compiled model extracted to: {compiled_extract_dir}")
print("\n📂 Compiled model structure:")
print_tree(compiled_extract_dir)

### Step 2.3: Integrate Manifest File

Copy the DDA-compatible manifest to the compiled model directory structure.

In [ ]:
# Copy DDA manifest to the model artifacts directory
manifest_destination = os.path.join(extract_dir_prefix, "manifest.json")
shutil.copy(new_manifest_path, manifest_destination)

print(f"{green_check_mark} Manifest integrated into model structure")
print(f"📍 Manifest location: {manifest_destination}")

### Step 2.4: Package for Greengrass Deployment

Create a ZIP archive of the complete model structure for Greengrass component deployment.

In [ ]:
# Package the complete model structure
folder_to_zip = extract_dir_prefix
uuid_for_model = str(uuid.uuid4()).split('-')[-1]
uuid_greengrass = uuid_for_model + "_greengrass_model_component"
output_zip = os.path.join(model_transformation_dir, uuid_greengrass)

print(f"📦 Creating Greengrass component package...")
print(f"🆔 Component ID: {uuid_greengrass}")
print("\n📂 Final directory structure:")
print_tree(folder_to_zip)

# Create ZIP archive
shutil.make_archive(output_zip, 'zip', folder_to_zip)
print(f"\n{green_check_mark} Component package created: {output_zip}.zip")

### Step 2.5: Configure S3 Upload Location

Specify where to store the packaged model artifacts for Greengrass component access.

In [ ]:
# Configure S3 storage location for model artifacts
print("🗂️ S3 Storage Configuration")
print("Configure where to store the packaged model artifacts:\n")

default_model_name = f'model-{uuid_for_model}'
s3_bucket_name = input(f"Model identifier (default: {default_model_name}): ") or default_model_name
s3_bucket_name = os.path.join(s3_bucket_name, uuid_greengrass + ".zip")

s3_uri = input("\nS3 base URI (e.g., s3://your-bucket/path/): ")
s3_uri = s3_uri.rstrip('/') + "/model_artifacts/"
artifact_upload_location = os.path.join(s3_uri, s3_bucket_name)

print(f"\n📍 Upload destination: {artifact_upload_location}")

### Step 2.6: Upload Model Artifacts

Upload the packaged model component to S3 for Greengrass deployment.

In [ ]:
# Upload packaged artifacts to S3
print(f"☁️ Uploading model artifacts to S3...")
print(f"📤 Destination: {artifact_upload_location}")

!aws s3 cp {output_zip + '.zip'} {artifact_upload_location}

print(f"{green_check_mark} Model artifacts uploaded successfully")

---

## Phase 3: Greengrass Component Creation

Create and deploy the AWS IoT Greengrass model component for edge inference.

### Step 3.1: Component Configuration

Define the Greengrass component parameters including name, version, and target platform.

In [ ]:
# Component configuration parameters
component_version_pattern = r'^\d+\.0+\.0+$'  # Format: x.0.0
platform_list = ["aarch64", "amd64"]

print("⚙️ Greengrass Component Configuration")
print("Configure the component details for deployment:\n")

# Component name validation
while True:
    component_name = input("Component name (format: model-*, e.g., model-defect-classifier): ")
    if component_name.startswith("model-"):
        print(f"{green_check_mark} Component name: {component_name}")
        break
    else:
        print(f"{error_mark} Invalid format. Use 'model-*' naming convention\n")

# Component version validation
while True:
    component_version = input("\nComponent version (format: x.0.0, e.g., 1.0.0): ")
    if re.match(component_version_pattern, component_version):
        print(f"{green_check_mark} Component version: {component_version}")
        break
    else:
        print(f"{error_mark} Invalid format. Use x.0.0 format\n")

# Friendly name for the model
model_friendly_name = input("\nFriendly name (optional, press Enter to use component name): ") or component_name
print(f"{green_check_mark} Friendly name: {model_friendly_name}")

# Model artifacts URI
model_artifacts_uri = input("\nModel artifacts URI (press Enter to use uploaded location): ") or artifact_upload_location
print(f"{green_check_mark} Artifacts URI: {model_artifacts_uri}")

# Target platform validation
while True:
    platform = input(f"\nTarget platform {platform_list}: ")
    if platform in platform_list:
        print(f"{green_check_mark} Target platform: {platform}")
        break
    else:
        print(f"{error_mark} Invalid platform. Choose from: {platform_list}\n")

### Step 3.2: Generate Component Recipe

Create the Greengrass component recipe with proper lifecycle management and dependencies.

In [ ]:
# Generate Greengrass component recipe
print("📋 Generating component recipe...")

# Extract model archive name for component paths
model_unarchived_path = model_artifacts_uri.split('/')[-1].split('.zip')[0]

# Set platform-specific DDA LocalServer component dependency
if platform == "aarch64":
    local_server_component = "aws.edgeml.dda.LocalServer.arm64"
elif platform == "amd64":
    local_server_component = "aws.edgeml.dda.LocalServer.amd64"
else:
    local_server_component = "aws.edgeml.dda.LocalServer"

print(f"🔗 DDA dependency: {local_server_component}")

# Construct the complete component recipe
recipe = {
    "RecipeFormatVersion": "2020-01-25",
    "ComponentName": component_name,
    "ComponentVersion": component_version,
    "ComponentType": "aws.greengrass.generic",
    "ComponentPublisher": "Amazon Lookout for Vision",
    
    # Component configuration
    "ComponentConfiguration": {
        "DefaultConfiguration": {
            "Autostart": False,
            "PYTHONPATH": "/usr/bin/python3.9",
            "ModelName": model_friendly_name
        }
    },
    
    # Dependencies on DDA LocalServer component
    "ComponentDependencies": {
        local_server_component: {
            "VersionRequirement": "^1.0.0",
            "DependencyType": "HARD"
        }
    },
    
    # Platform-specific manifests
    "Manifests": [
        {
            "Platform": {
                "os": "linux",
                "architecture": platform
            },
            
            # Lifecycle management
            "Lifecycle": {
                "Startup": {
                    "Script": f"python3 /aws_dda/model_convertor.py --unarchived_model_path {{artifacts:decompressedPath}}/{model_unarchived_path}/ --model_version {component_version} --model_name {component_name}",
                    "Timeout": 900,
                    "requiresPrivilege": True,
                    "runWith": {
                        "posixUser": "root"
                    }
                },
                "Shutdown": {
                    "Script": f"python3 /aws_dda/convert_model_cleanup.py --model_name {component_name}",
                    "Timeout": 900,
                    "requiresPrivilege": True,
                    "runWith": {
                        "posixUser": "root"
                    }
                }
            },
            
            # Model artifacts configuration
            "Artifacts": [
                {
                    "Uri": model_artifacts_uri,
                    "Digest": "",  # Will be calculated by Greengrass
                    "Algorithm": "SHA-256",
                    "Unarchive": "ZIP",
                    "Permission": {
                        "Read": "ALL",
                        "Execute": "ALL"
                    }
                }
            ]
        }
    ],
    "Lifecycle": {}
}

print(f"{green_check_mark} Component recipe generated")
print(f"📦 Component: {component_name} v{component_version}")
print(f"🏗️ Platform: {platform}")
print(f"📍 Artifacts: {model_artifacts_uri}")

### Step 3.3: Deploy Greengrass Component

Create the Greengrass component and monitor its deployment status.

In [ ]:
# Create and deploy the Greengrass component
client = boto3.client('greengrassv2')

print("🚀 Creating Greengrass model component...")
print(f"📋 Recipe: {json.dumps(recipe, indent=2)}\n")

try:
    # Create the component version
    response = client.create_component_version(
        inlineRecipe=json.dumps(recipe)
    )
    
    model_component_arn = response.get('arn')
    print(f"{green_check_mark} Component created successfully!")
    print(f"🆔 Component ARN: {model_component_arn}")
    
    # Monitor component status
    print("\n⏳ Monitoring component status...")
    
    while True:
        print("🔍 Checking component status...")
        
        response = client.describe_component(arn=model_component_arn)
        status = response['status']['componentState']
        message = response['status'].get('message', 'No additional message')
        errors = response['status'].get('errors', [])
        
        # Check if component is ready
        if status not in ["REQUESTED", "IN_PROGRESS"]:
            if status == "DEPLOYABLE":
                print(f"\n{green_check_mark} Component is ready for deployment!")
                print(f"📊 Final Status: {status}")
            else:
                print(f"\n⚠️ Component Status: {status}")
            
            # Display additional details if available
            if message != 'NONE' and message != 'No additional message':
                print(f"💬 Message: {message}")
            if errors:
                print(f"❌ Errors: {errors}")
            break
        else:
            print(f"⏳ Status: {status} - Checking again in 5 seconds...")
            time.sleep(5)
            
except Exception as e:
    print(f"{error_mark} Component creation failed: {str(e)}")
    traceback.print_exc()

---

## Summary

🎉 **Greengrass Model Component Setup Complete!**

### What was accomplished:

1. **✅ Model Processing**: Downloaded and processed SageMaker-trained models
2. **✅ Manifest Creation**: Generated DDA-compatible manifest files
3. **✅ Artifact Packaging**: Created Greengrass-ready component packages
4. **✅ Component Deployment**: Successfully deployed to AWS IoT Greengrass

### Component Details:
- **Name**: `{component_name if 'component_name' in locals() else 'N/A'}`
- **Version**: `{component_version if 'component_version' in locals() else 'N/A'}`
- **Platform**: `{platform if 'platform' in locals() else 'N/A'}`
- **Model Type**: `{model_name if 'model_name' in locals() else 'N/A'}`

### Next Steps:
1. **Deploy to Edge Device**: Use AWS IoT Greengrass console to deploy the component
2. **Configure DDA**: Set up the Defect Detection Application to use this model
3. **Test Inference**: Validate model performance on edge device
4. **Monitor Performance**: Use CloudWatch for component monitoring

### Useful Commands:
```bash
# List components
aws greengrassv2 list-components

# Create deployment
aws greengrassv2 create-deployment --target-arn <thing-arn> --components '{"<component-name>":{"componentVersion":"<version>"}}'
```